# 実行環境の作成

## Google Drive

Google Driveをマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Google Sheets

ライブラリのインストール

In [2]:
!pip install --upgrade -q gspread

     |████████████████████████████████| 40 kB 3.6 MB/s 


Google Sheetsの認証

In [3]:
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

## Google Cloud Storage

下記コードでGCPに接続

In [4]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [5]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

deb http://packages.cloud.google.com/apt gcsfuse-bionic main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  68567      0 --:--:-- --:--:-- --:--:-- 68567
OK
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease [5,391 B]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.c

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [6]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

2022/11/20 00:35:14.622144 Start gcsfuse/0.41.8 (Go version go1.18.4) for app "" using mount point: /content/statistics-hyogo
2022/11/20 00:35:14.642495 Opening GCS connection...
2022/11/20 00:35:15.876099 Mounting file system "statistics-hyogo"...
2022/11/20 00:35:15.879556 File system has been successfully mounted.


# Rechartsデータ作成

## BigQueryからデータ取得

In [7]:
import pandas as pd

def get_gbq(table_id):
  project_id='primal-buttress-342908'
  dataset_id = 'contents'  
  query = f'SELECT * FROM `{project_id}.{dataset_id}.{table_id}`'
  
  return pd.read_gbq(query, project_id, dialect='standard')

In [8]:
df = get_gbq('total-population-prefecture')
df = df[df['areaCode'] == '28000']
df

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,value,unit,index,prefCode,prefName,cityCode,cityName,bigCityFlag,areaName,governmentType,rankJapan
1215,0000010101,社会・人口統計体系,A1101,総人口,1975,1975年度,28000,4992140.0,人,27,28,兵庫県,None,None,None,兵庫県,prefecture,6
1216,0000010101,社会・人口統計体系,A1101,総人口,2008,2008年度,28000,5592000.0,人,27,28,兵庫県,None,None,None,兵庫県,prefecture,7
1217,0000010101,社会・人口統計体系,A1101,総人口,1997,1997年度,28000,5455000.0,人,27,28,兵庫県,None,None,None,兵庫県,prefecture,8
1218,0000010101,社会・人口統計体系,A1101,総人口,1989,1989年度,28000,5372000.0,人,27,28,兵庫県,None,None,None,兵庫県,prefecture,8
1219,0000010101,社会・人口統計体系,A1101,総人口,1999,1999年度,28000,5528000.0,人,27,28,兵庫県,None,None,None,兵庫県,prefecture,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5485,0000010101,社会・人口統計体系,A110102,総人口（女）,1976,1976年度,28000,2561000.0,人,27,28,兵庫県,None,None,None,兵庫県,prefecture,6
5486,0000010101,社会・人口統計体系,A110102,総人口（女）,1988,1988年度,28000,2751000.0,人,27,28,兵庫県,None,None,None,兵庫県,prefecture,7
5487,0000010101,社会・人口統計体系,A110102,総人口（女）,2015,2015年度,28000,2893239.0,人,27,28,兵庫県,None,None,None,兵庫県,prefecture,7
5488,0000010101,社会・人口統計体系,A110102,総人口（女）,1985,1985年度,28000,2710236.0,人,27,28,兵庫県,None,None,None,兵庫県,prefecture,7


## rechartsのtimeChartに整形する関数

In [159]:
def recharts_timedata(cardId,areaCode):

  # カード情報の取得
  card = get_statistics_card(cardId)
  print(card)

  # 地域情報の取得
  area = get_area(areaCode)

  # GBQからデータ取得
  df = get_gbq(cardId)
  df = df[df['areaCode'] == areaCode]

  # recharsデータの作成
  categories = df['categoryName'].drop_duplicates()
  df_res = pd.DataFrame(index=[], columns=['time'])
  for c in categories:
    df_c = df[df.copy()['categoryName'] == c][['timeCode','value']].sort_values('timeCode').rename(columns={'value': c, 'timeCode': 'time'})
    df_res = pd.merge(df_res, df_c,on='time', how='outer')

  # 最新データの取得
  category = card['categories'][0]
  latestData = df[df.copy()['categoryCode'] == category['categoryCode']].sort_values('timeCode', ascending=False).head(1).to_dict(orient='records')[0]
  print(latestData)

  r_dict = {
      'cardTitle':area['areaName'] + card['cardTitle'].replace('都道府県', '').replace('市区町村', ''),
      'infoTitle':latestData['timeName'],
      'chartData':df_res.to_dict(orient='records')
  }

  return r_dict

In [160]:
res = recharts_timedata('total-population-prefecture','28000')
res

{'index': 0, 'cardId': 'total-population-prefecture', 'cardTitle': '都道府県の総人口', 'cardIndex': 1, 'governmentType': 'prefecture', 'menuId': 'population', 'menuTitle': '人口', 'fieldId': 'population', 'fieldTitle': '人口・世帯', 'chartComponent': 'TimeChart', 'categories': [{'categoryCode': 'A1101', 'categoryName': '総人口', 'isSelect': 'TRUE', 'type': 'column', 'yAxis': 0}, {'categoryCode': 'A110101', 'categoryName': '総人口（男）', 'isSelect': '', 'type': 'column', 'yAxis': 0}, {'categoryCode': 'A110102', 'categoryName': '総人口（女）', 'isSelect': '', 'type': 'column', 'yAxis': 0}], 'estatParams': {'statsDataId': '0000010101', 'cdCat01': 'A1101,A110101,A110102'}}
{'statsDataId': '0000010101', 'statsDataName': '社会・人口統計体系', 'categoryCode': 'A1101', 'categoryName': '総人口', 'timeCode': '2019', 'timeName': '2019年度', 'areaCode': '28000', 'value': 5466000.0, 'unit': '人', 'index': 27, 'prefCode': 28, 'prefName': '兵庫県', 'cityCode': None, 'cityName': None, 'bigCityFlag': None, 'areaName': '兵庫県', 'governmentType': 'pref

{'cardTitle': '兵庫県の総人口',
 'infoTitle': '2019年度',
 'chartData': [{'time': '1975',
   '総人口': 4992140.0,
   '総人口（男）': 2453277.0,
   '総人口（女）': 2538863.0},
  {'time': '1976', '総人口': 5029000.0, '総人口（男）': 2468000.0, '総人口（女）': 2561000.0},
  {'time': '1977', '総人口': 5063000.0, '総人口（男）': 2481000.0, '総人口（女）': 2582000.0},
  {'time': '1978', '総人口': 5093000.0, '総人口（男）': 2493000.0, '総人口（女）': 2600000.0},
  {'time': '1979', '総人口': 5119000.0, '総人口（男）': 2502000.0, '総人口（女）': 2616000.0},
  {'time': '1980', '総人口': 5144892.0, '総人口（男）': 2512358.0, '総人口（女）': 2632534.0},
  {'time': '1981', '総人口': 5171000.0, '総人口（男）': 2523000.0, '総人口（女）': 2648000.0},
  {'time': '1982', '総人口': 5198000.0, '総人口（男）': 2534000.0, '総人口（女）': 2664000.0},
  {'time': '1983', '総人口': 5227000.0, '総人口（男）': 2547000.0, '総人口（女）': 2680000.0},
  {'time': '1984', '総人口': 5252000.0, '総人口（男）': 2557000.0, '総人口（女）': 2695000.0},
  {'time': '1985', '総人口': 5278050.0, '総人口（男）': 2567814.0, '総人口（女）': 2710236.0},
  {'time': '1986', '総人口': 5300000.0, '総人口（男）': 25

## 統計カード一覧

In [9]:
import json

def get_statistics_cardlist():
  with open('/content/drive/MyDrive/statistics-hyogo/contents/statistics-cards.json') as j:
    cardlist = json.load(j)

  return cardlist

In [10]:
cardlist = get_statistics_cardlist()
print(cardlist)

[{'index': 0, 'cardId': 'total-population-prefecture', 'cardTitle': '都道府県の総人口', 'cardIndex': 1, 'governmentType': 'prefecture', 'menuId': 'population', 'menuTitle': '人口', 'fieldId': 'population', 'fieldTitle': '人口・世帯', 'chartComponent': 'TimeChart', 'categories': [{'categoryCode': 'A1101', 'categoryName': '総人口', 'isSelect': 'TRUE', 'type': 'column', 'yAxis': 0}, {'categoryCode': 'A110101', 'categoryName': '総人口（男）', 'isSelect': '', 'type': 'column', 'yAxis': 0}, {'categoryCode': 'A110102', 'categoryName': '総人口（女）', 'isSelect': '', 'type': 'column', 'yAxis': 0}], 'estatParams': {'statsDataId': '0000010101', 'cdCat01': 'A1101,A110101,A110102'}}, {'index': 1, 'cardId': 'japanese-population-prefecture', 'cardTitle': '都道府県の日本人人口', 'cardIndex': '', 'governmentType': 'prefecture', 'menuId': 'population', 'menuTitle': '人口', 'fieldId': 'population', 'fieldTitle': '人口・世帯', 'chartComponent': 'TimeChart', 'categories': [{'categoryCode': 'A1102', 'categoryName': '日本人人口', 'isSelect': 'TRUE', 'type': 

## カード情報を取得する関数

In [107]:
def get_statistics_card(cardId):
  with open('/content/drive/MyDrive/statistics-hyogo/contents/statistics-cards.json') as j:
    cardlist = json.load(j)

    card = list(filter(lambda item : item['cardId'] == cardId, cardlist))
    return card[0]

In [108]:
card = get_statistics_card('total-population-prefecture')
print(card)

{'index': 0, 'cardId': 'total-population-prefecture', 'cardTitle': '都道府県の総人口', 'cardIndex': 1, 'governmentType': 'prefecture', 'menuId': 'population', 'menuTitle': '人口', 'fieldId': 'population', 'fieldTitle': '人口・世帯', 'chartComponent': 'TimeChart', 'categories': [{'categoryCode': 'A1101', 'categoryName': '総人口', 'isSelect': 'TRUE', 'type': 'column', 'yAxis': 0}, {'categoryCode': 'A110101', 'categoryName': '総人口（男）', 'isSelect': '', 'type': 'column', 'yAxis': 0}, {'categoryCode': 'A110102', 'categoryName': '総人口（女）', 'isSelect': '', 'type': 'column', 'yAxis': 0}], 'estatParams': {'statsDataId': '0000010101', 'cdCat01': 'A1101,A110101,A110102'}}


## 地域一覧

In [61]:
def get_arealist():
  with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    arealist = json.load(j)
    
  return arealist

In [62]:
arealist = get_arealist()f
preflist = list(filter(lambda x: x['governmentType'] == 'prefecture', arealist))
print(preflist)

[{'index': 0, 'prefCode': 1, 'prefName': '北海道', 'cityCode': None, 'cityName': None, 'bigCityFlag': None, 'areaCode': '01000', 'areaName': '北海道', 'governmentType': 'prefecture'}, {'index': 1, 'prefCode': 2, 'prefName': '青森県', 'cityCode': None, 'cityName': None, 'bigCityFlag': None, 'areaCode': '02000', 'areaName': '青森県', 'governmentType': 'prefecture'}, {'index': 2, 'prefCode': 3, 'prefName': '岩手県', 'cityCode': None, 'cityName': None, 'bigCityFlag': None, 'areaCode': '03000', 'areaName': '岩手県', 'governmentType': 'prefecture'}, {'index': 3, 'prefCode': 4, 'prefName': '宮城県', 'cityCode': None, 'cityName': None, 'bigCityFlag': None, 'areaCode': '04000', 'areaName': '宮城県', 'governmentType': 'prefecture'}, {'index': 4, 'prefCode': 5, 'prefName': '秋田県', 'cityCode': None, 'cityName': None, 'bigCityFlag': None, 'areaCode': '05000', 'areaName': '秋田県', 'governmentType': 'prefecture'}, {'index': 5, 'prefCode': 6, 'prefName': '山形県', 'cityCode': None, 'cityName': None, 'bigCityFlag': None, 'areaCode'

## 地域を取得する関数

In [113]:
def get_area(areaCode):
  with open('drive/MyDrive/statistics-hyogo/resas/arealist.json') as j:
    arealist = json.load(j)
    
    area = list(filter(lambda x: x['areaCode'] == areaCode, arealist))
    return area[0]

In [114]:
area = get_area('28000')
print(area)

{'index': 27, 'prefCode': 28, 'prefName': '兵庫県', 'cityCode': None, 'cityName': None, 'bigCityFlag': None, 'areaCode': '28000', 'areaName': '兵庫県', 'governmentType': 'prefecture'}


## jsonに保存

In [92]:
for card in cardlist:
  cardId = card['cardId']
  governmentType = card['governmentType']
  categories = card['categories']

  print(card)

  if governmentType == 'prefecture':
    preflist = list(filter(lambda x: x['governmentType'] == 'prefecture', arealist))

    for pref in preflist:
      areaCode = pref['areaCode']
      
      df = recharts_timedata(cardId,areaCode)

      d = df.to_dict()
      print(d)


{'index': 0, 'cardId': 'total-population-prefecture', 'cardTitle': '都道府県の総人口', 'cardIndex': 1, 'governmentType': 'prefecture', 'menuId': 'population', 'menuTitle': '人口', 'fieldId': 'population', 'fieldTitle': '人口・世帯', 'chartComponent': 'TimeChart', 'categories': [{'categoryCode': 'A1101', 'categoryName': '総人口', 'isSelect': 'TRUE', 'type': 'column', 'yAxis': 0}, {'categoryCode': 'A110101', 'categoryName': '総人口（男）', 'isSelect': '', 'type': 'column', 'yAxis': 0}, {'categoryCode': 'A110102', 'categoryName': '総人口（女）', 'isSelect': '', 'type': 'column', 'yAxis': 0}], 'estatParams': {'statsDataId': '0000010101', 'cdCat01': 'A1101,A110101,A110102'}}
{'time': {0: '1975', 1: '1976', 2: '1977', 3: '1978', 4: '1979', 5: '1980', 6: '1981', 7: '1982', 8: '1983', 9: '1984', 10: '1985', 11: '1986', 12: '1987', 13: '1988', 14: '1989', 15: '1990', 16: '1991', 17: '1992', 18: '1993', 19: '1994', 20: '1995', 21: '1996', 22: '1997', 23: '1998', 24: '1999', 25: '2000', 26: '2001', 27: '2002', 28: '2003', 29:

KeyboardInterrupt: ignored